In [16]:
import requests
from bs4 import BeautifulSoup
import random
import time
from itertools import chain
import polars as pl

In [17]:
URL = "https://www.boulanger.com/c/nav-filtre/smartphone-telephone-portable?_etat_produit~neuf"
URL_INIT = "https://www.boulanger.com"

In [14]:
def create_session() -> requests.Session:
    """`create_session`: Create a requests session with a custom header to make the server believe the scraper is actually a browser opened by a real user.
    
    `Returns`
    --------- ::
    
        requests.Session
    
    `Example(s)`
    ---------
    
    >>> create_session()
    ... <requests.sessions.Session at 0x223e7834df0>"""
    session = requests.Session()
    session.headers.update(
        {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"
        }
    )
    return session

In [5]:
def random_timer():
    random_delay = random.uniform(0.1, 2)
    return time.sleep(random_delay)

In [51]:
def create_url_list(URL: str, nb_page: int) -> list[str]:
    url_list = list()
    url_list.append(URL)

    # nb_page = 18
    for index_page in range(2, nb_page):
        page_url = f"{URL}&numPage={index_page}"
        url_list.append(page_url)
    return url_list

In [52]:
def read_pages(url_list: list[str], session: requests.Session) -> list[str]:
    list_pages = list()
    for page_number, url in enumerate(url_list):
        random_timer()
        response = session.get(url)
        content = response.text
        list_pages.append(content)
        if response.status_code == 200:
            print(f"Page {page_number+1} -> Successfull Extraction.")
        else:
            print(f"Page {page_number+1} -> Unsuccessfull Extraction.")
    return list_pages

In [53]:
def extract_all_urls(pages:list[str]) -> list[str]:
    links_list = list()
    for page in pages:
        soup = BeautifulSoup(page, "html.parser")
        links = soup.find_all("a", attrs={"class": "product-list__product-image-link"})
        links_list.append(links)
    link_matches = list(chain.from_iterable(links_list))
    all_urls = [URL_INIT + link.get("href") for link in link_matches]
    random.shuffle(all_urls) # mélange aléatoire
    return all_urls

In [54]:
def extract_all_pages(all_urls: list[str], session: requests.Session):
    product_pages = list()
    for product_number, url in enumerate(all_urls):
        random_timer()
        response = session.get(url)
        content_page = response.text
        product_pages.append(content_page)
        if response.status_code == 200:
            print(f"Product {product_number+1} -> Successfull Extraction.")
        else:
            print(f"Product {product_number+1} -> Unsuccessfull Extraction.")
    return product_pages

In [55]:
session = create_session()
url_list = create_url_list(URL, 19)

In [18]:
url = "https://www.boulanger.com/ref/1189138"

In [22]:
#response = session.get(url)
#content = response.text
#soup = BeautifulSoup(content, "html.parser")
#str(soup.find("a", attrs={"class": "rating"}))

In [56]:
pages = read_pages(url_list=url_list, session=session)
all_urls = extract_all_urls(pages)

Page 1 -> Successfull Extraction.


In [57]:
product_pages = extract_all_pages(all_urls, session)

Product 1 -> Successfull Extraction.
Product 2 -> Successfull Extraction.
Product 3 -> Successfull Extraction.
Product 4 -> Successfull Extraction.
Product 5 -> Successfull Extraction.
Product 6 -> Successfull Extraction.
Product 7 -> Successfull Extraction.
Product 8 -> Successfull Extraction.
Product 9 -> Successfull Extraction.
Product 10 -> Successfull Extraction.
Product 11 -> Successfull Extraction.
Product 12 -> Successfull Extraction.
Product 13 -> Successfull Extraction.
Product 14 -> Successfull Extraction.
Product 15 -> Successfull Extraction.
Product 16 -> Successfull Extraction.
Product 17 -> Successfull Extraction.
Product 18 -> Successfull Extraction.
Product 19 -> Successfull Extraction.
Product 20 -> Successfull Extraction.
Product 21 -> Successfull Extraction.
Product 22 -> Successfull Extraction.
Product 23 -> Successfull Extraction.
Product 24 -> Successfull Extraction.
Product 25 -> Successfull Extraction.
Product 26 -> Successfull Extraction.
Product 27 -> Success

In [3]:
#response = session.get(URL)
#content = response.text
#soup = BeautifulSoup(content, "html.parser")
#links = soup.find_all("a", attrs={"class": "product-list__product-image-link"})
#all_urls = [URL_INIT + link.get("href") for link in links]
#random.shuffle(all_urls) # mélange aléatoire de la liste des url

In [4]:
#product_pages = list()
#for url in all_urls:
#    random_delay = random.uniform(0.25, 2)
#    time.sleep(random_delay)
#    response = session.get(url)
#    content_page = response.text
#    product_pages.append(content_page)
#    print("succès extraction")

In [3]:
#soup = BeautifulSoup(product_pages[0], "html.parser")

In [58]:
def _extract_nb_reviews_V0(soup: BeautifulSoup) -> str:
    nb_reviews = soup.find("span", attrs={"class": "rating__count"}).text
    return nb_reviews

In [59]:
def _extract_stars_V0(soup: BeautifulSoup) -> str:
    stars = str(soup.find("a", attrs={"class": "rating"}))
    return stars

In [60]:
def _extract_price_V0(soup: BeautifulSoup) -> str:
    price = soup.find("p", attrs={"class": "price__amount"}).text.strip()
    return price

In [111]:
def extract_features(soup: BeautifulSoup) -> dict:
    span_elements = soup.find_all("span", class_="is--medium")
    keys = [span_element.text[:-2] for span_element in span_elements]
    # on retire les 2 caractères " :" pour faire les clés du futur dictionnaire
    keys.extend(["Prix", "Etoiles", "Commentaires"])

    feature_values = list()
    for span_element in span_elements:
        feature_values.append(str(span_element.find_next_sibling(text=True)).strip())
    
    feature_values.extend([_extract_price_V0(soup), _extract_stars_V0(soup), _extract_nb_reviews_V0(soup)])

    characteristics = {column: feature for column, feature in zip(keys, feature_values)}
    return characteristics

In [103]:
def extract_model(dict_features: dict):
    try:
        model = dict_features["Modèle"]
    except KeyError:
        return None
    else:
        return model


def extract_color(dict_features: dict):
    try:
        color = dict_features["Couleur"]
    except KeyError:
        return None
    else:
        return color


def extract_os(dict_features: dict):
    try:
        os = dict_features["Système d'exploitation"]
    except KeyError:
        return None
    else:
        return os


def extract_sim_type(dict_features: dict):
    try:
        sim_type = dict_features["Type de SIM"]
    except KeyError:
        return None
    else:
        return sim_type


def extract_sim_number(dict_features: dict):
    try:
        sim_number = dict_features["Nombre de SIM"]
    except KeyError:
        return None
    else:
        return sim_number


def extract_cpu(dict_features: dict):
    try:
        cpu = dict_features["Processeur"]
    except KeyError:
        return None
    else:
        return cpu


def extract_cpu_details(dict_features: dict):
    try:
        cpu_details = dict_features["Détails du Processeur"]
    except KeyError:
        return None
    else:
        return cpu_details


def extract_battery(dict_features: dict):
    try:
        battery = dict_features["Batterie"]
    except KeyError:
        return None
    else:
        return battery


def extract_charging_type(dict_features: dict):
    try:
        charging_type = dict_features["Type de charge"]
    except KeyError:
        return None
    else:
        return charging_type

def extract_fast_charging(dict_features: dict):
    try:
        fast_charging = dict_features["Charge rapide"]
    except KeyError:
        return None
    else:
        return fast_charging

def extract_screen_size(dict_features: dict):
    try:
        screen_size = dict_features["Taille de l'écran (diagonale, en pouces)"]
    except KeyError:
        return None
    else:
        return screen_size

def extract_screen_tech(dict_features: dict):
    try:
        screen_tech = dict_features["Technologie de l'écran"]
    except KeyError:
        return None
    else:
        return screen_tech

def extract_screen_resolution(dict_features: dict):
    try:
        screen_resolution = dict_features["Résolution de l'écran"]
    except KeyError:
        return None
    else:
        return screen_resolution

def extract_screen_type(dict_features: dict):
    try:
        screen_type = dict_features["Type d'écran"]
    except KeyError:
        return None
    else:
        return screen_type

def extract_refresh_rate(dict_features: dict):
    try:
        refresh_rate = dict_features["Fréquence de balayage"]
    except KeyError:
        return None
    else:
        return refresh_rate

def extract_network(dict_features: dict):
    try:
        network = dict_features["Réseau"]
    except KeyError:
        return None
    else:
        return network

def extract_bluetooth(dict_features: dict):
    try:
        bluetooth = dict_features["Bluetooth"]
    except KeyError:
        return None
    else:
        return bluetooth

def extract_wifi(dict_features: dict):
    try:
        wifi = dict_features["Norme Wifi"]
    except KeyError:
        return None
    else:
        return wifi

def extract_usb_type_c(dict_features: dict):
    try:
        usb_type_c = dict_features["Port USB Type-C"]
    except KeyError:
        return None
    else:
        return usb_type_c

def extract_storage(dict_features: dict):
    try:
        storage = dict_features["Mémoire interne"]
    except KeyError:
        return None
    else:
        return storage

def extract_upgrade_storage(dict_features: dict):
    try:
        upgrade_storage = dict_features["Extension de la mémoire"]
    except KeyError:
        return None
    else:
        return upgrade_storage

def extract_ram(dict_features: dict):
    try:
        ram = dict_features["Mémoire RAM"]
    except KeyError:
        return None
    else:
        return ram

def extract_sensor(dict_features: dict):
    try:
        sensor = dict_features["Nombre de capteurs"]
    except KeyError:
        return None
    else:
        return sensor

def extract_sensor_resolution(dict_features: dict):
    try:
        sensor_resolution = dict_features["Résolution"]
    except KeyError:
        return None
    else:
        return sensor_resolution

def extract_repairability_index(dict_features: dict):
    try:
        repairability_index = dict_features["Indice de réparabilité"]
    except KeyError:
        return None
    else:
        return repairability_index

def extract_warranty(dict_features: dict):
    try:
        warranty = dict_features["Garantie"]
    except KeyError:
        return None
    else:
        return warranty

def extract_made_in(dict_features: dict):
    try:
        made_in = dict_features["Fabriqué en"]
    except KeyError:
        return None
    else:
        return made_in

def extract_brand(dict_features: dict):
    try:
        brand = dict_features["Marque"]
    except KeyError:
        return None
    else:
        return brand

def extract_das_head(dict_features: dict):
    try:
        das_head = dict_features["DAS Tête"]
    except KeyError:
        return None
    else:
        return das_head

def extract_das_chest(dict_features: dict):
    try:
        das_chest = dict_features["DAS Tronc"]
    except KeyError:
        return None
    else:
        return das_chest

def extract_das_limb(dict_features: dict):
    try:
        das_limb = dict_features["DAS Membres"]
    except KeyError:
        return None
    else:
        return das_limb

def extract_height(dict_features: dict):
    try:
        height = dict_features["Hauteur produit"]
    except KeyError:
        return None
    else:
        return height

def extract_width(dict_features: dict):
    try:
        width = dict_features["Largeur produit"]
    except KeyError:
        return None
    else:
        return width

def extract_thickness(dict_features: dict):
    try:
        thickness = dict_features["Epaisseur produit"]
    except KeyError:
        return None
    else:
        return thickness

def extract_net_weight(dict_features: dict):
    try:
        net_weight = dict_features["Poids net"]
    except KeyError:
        return None
    else:
        return net_weight

def extract_price(dict_features: dict):
    try:
        price = dict_features["Prix"]
    except KeyError:
        return None
    else:
        return price

def extract_stars(dict_features: dict):
    try:
        stars = dict_features["Etoiles"]
    except KeyError:
        return None
    else:
        return stars

def extract_reviews(dict_features: dict):
    try:
        reviews = dict_features["Commentaires"]
    except KeyError:
        return None
    else:
        return reviews

In [104]:
from dataclasses import dataclass
from serde import serialize
from serde.json import to_json

@serialize
@dataclass
class Smartphone:
    """This dataclass represents all features of a Smartphone."""

    model: str
    color: str
    os: str
    sim_type: str
    sim_number: str
    cpu: str
    cpu_details: str
    battery: str
    charging_type: str
    fast_charging: str
    screen_size: str
    screen_tech: str
    screen_resolution: str
    screen_type: str
    refresh_rate: str
    network: str
    bluetooth: str
    wifi: str
    usb_type_c: str
    storage: str
    upgrade_storage: str
    ram: str
    sensor: str
    sensor_resolution: str
    repairability_index: str
    warranty: str
    made_in: str
    brand: str
    das_head: str
    das_chest: str
    das_limbs: str
    height: str
    width: str
    thickness: str
    net_weight: str
    price: str
    stars: str
    reviews: str

In [110]:
def smartphone_characteristics(dict_features: dict) -> Smartphone:
    model = extract_model(dict_features)
    color = extract_color(dict_features)
    os = extract_os(dict_features)
    sim_type = extract_sim_type(dict_features)
    sim_number = extract_sim_number(dict_features)
    cpu = extract_cpu(dict_features)
    cpu_details = extract_cpu_details(dict_features)
    battery = extract_battery(dict_features)
    charging_type = extract_charging_type(dict_features)
    fast_charging = extract_fast_charging(dict_features)
    screen_size = extract_screen_size(dict_features)
    screen_tech = extract_screen_tech(dict_features)
    screen_resolution = extract_screen_resolution(dict_features)
    screen_type = extract_screen_type(dict_features)
    refresh_rate = extract_refresh_rate(dict_features)
    network = extract_network(dict_features)
    bluetooth = extract_bluetooth(dict_features)
    wifi = extract_wifi(dict_features)
    usb_type_c = extract_usb_type_c(dict_features)
    storage = extract_storage(dict_features)
    upgrade_storage = extract_upgrade_storage(dict_features)
    ram = extract_ram(dict_features)
    sensor = extract_sensor(dict_features)
    sensor_resolution = extract_sensor_resolution(dict_features)
    repairability_index = extract_repairability_index(dict_features)
    warranty = extract_warranty(dict_features)
    made_in = extract_made_in(dict_features)
    brand = extract_brand(dict_features)
    das_head = extract_das_head(dict_features)
    das_chest = extract_das_chest(dict_features)
    das_limbs = extract_das_limb(dict_features)
    height = extract_height(dict_features)
    width = extract_width(dict_features)
    thickness = extract_thickness(dict_features)
    net_weight = extract_net_weight(dict_features)
    price = extract_price(dict_features)
    stars = extract_stars(dict_features)
    reviews = extract_reviews(dict_features)
    
    return Smartphone(
        model=model,
        color=color,
        os=os,
        sim_type=sim_type,
        sim_number=sim_number,
        cpu=cpu,
        cpu_details=cpu_details,
        battery=battery,
        charging_type=charging_type,
        fast_charging=fast_charging,
        screen_size=screen_size,
        screen_tech=screen_tech,
        screen_resolution=screen_resolution,
        screen_type=screen_type,
        refresh_rate=refresh_rate,
        network=network,
        bluetooth=bluetooth,
        wifi=wifi,
        usb_type_c=usb_type_c,
        storage=storage,
        upgrade_storage=upgrade_storage,
        ram=ram,
        sensor=sensor,
        sensor_resolution=sensor_resolution,
        repairability_index=repairability_index,
        warranty=warranty,
        made_in=made_in,
        brand=brand,
        das_head=das_head,
        das_chest=das_chest,
        das_limbs=das_limbs,
        height=height,
        width=width,
        thickness=thickness,
        net_weight=net_weight,
        price=price,
        stars=stars,
        reviews=reviews
    )

In [106]:
smartphone_list = list()
for article in product_pages:
    soup = BeautifulSoup(article, "html.parser")
    dict_features = extract_features(soup)
    smartphone = smartphone_characteristics(dict_features)
    smartphone_list.append(smartphone)

In [39]:
smartphones_json = to_json(smartphone_list)

In [40]:
file_path = "smartphones.json"

with open(file_path, "w",  encoding='utf-8') as json_file:
    json_file.write(smartphones_json)

In [104]:
df = pl.read_json("data/smartphones.json")

Note **TODO** : Récupérér le nombre d'avis et la note moyenne, qui va servir d'indicateur à l'achat

- Type d'écran (plat, incurvé) à récupérer aussi
- Plus de détails sur l'appareil photo aussi, résolution, type capteur.
- Avec toutes ces caractéristiques, on évite au maximum le biais de variable omise !

In [105]:
df.select(pl.all().is_null().sum()) # pour voir combien de nulls il y a sur chaque colonne

repairability_index,fast_charging,color,os,usb_type_c,sensor,made_in,net_weight,charging_type,das_limbs,height,brand,upgrade_storage,warranty,thickness,refresh_rate,bluetooth,width,battery,model,cpu_details,das_chest,sim_number,ram,screen_tech,sim_type,network,storage,cpu,screen_size,screen_resolution,das_head,price
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
16,2,1,2,86,81,48,19,17,13,4,0,1,93,4,338,0,4,24,0,211,13,6,29,2,0,0,0,5,0,0,13,0


In [106]:
df = df.drop("warranty", "refresh_rate", "cpu_details")
df = df.filter(pl.col("repairability_index").is_not_null())
df = df.filter((pl.col("ram").is_not_null()) & (pl.col("ram") != "Non communiqué"))

In [107]:
df.group_by("usb_type_c").count().sort("count", descending=True)

usb_type_c,count
str,u32
"""Oui""",361
null,52
"""Non""",49


Tous les téléphones *Apple* ont un usb_type_c = `null` car ils n'en ont tout simplement pas !

In [117]:
df.filter(pl.col("usb_type_c").is_null()).filter(pl.col("brand") != "APPLE")

repairability_index,fast_charging,color,os,usb_type_c,sensor,made_in,net_weight,charging_type,das_limbs,height,brand,upgrade_storage,thickness,bluetooth,width,battery,model,das_chest,sim_number,ram,screen_tech,sim_type,network,storage,cpu,screen_size,screen_resolution,das_head,price
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""8,5 /10""","""Oui""","""Noir""","""Android 11""",null,"""3""","""Chine""","""182,00 g""","""Pas de charge …","""2,776 W/kg""","""156,80 mm""","""OPPO""","""Non""","""7,60 mm""","""5.2""","""72,10 mm""","""4300 mAh""","""OPPO Reno 6""","""1,175 W/kg""","""2 SIM""","""8 Go""","""AMOLED Rigide""","""Nano SIM (x2)""","""5G""","""128 Go""","""8 coeurs jusqu…","""6,4"" soit 16,3…","""2400 x 1080 pi…","""0,957 W/kg""","""281,83€"""
"""8,5 /10""","""Oui""","""Bleu""","""Android 11""",null,"""3""","""Chine""","""182,00 g""","""Pas de charge …","""2,776 W/kg""","""156,80 mm""","""OPPO""","""Non""","""7,60 mm""","""5.2""","""72,10 mm""","""4300 mAh""","""OPPO Reno 6""","""1,175 W/kg""","""2 SIM""","""8 Go""","""AMOLED Rigide""","""Nano SIM (x2)""","""5G""","""128 Go""","""8 coeurs jusqu…","""6,4"" soit 16,3…","""2400 x 1080 pi…","""0,957 W/kg""","""277,56€"""
"""7,4 /10""","""Oui""","""Noir""","""Android 12""",null,"""2""","""Chine""","""192,00 g""","""Pas de charge …","""2,541 W/kg""","""164,90 mm""","""XIAOMI""","""Micro SD""","""9,10 mm""","""5""","""76,80 mm""","""5000 mAh""","""Xiaomi Redmi A…","""0,997 W/kg""","""2 SIM""","""2 Go""","""HD+""","""Nano SIM (x2) …","""4G""","""32 Go""","""4 coeurs jusqu…","""6,5"" soit 16,5…","""1600 x 720 pix…","""0,846 W/kg""","""91,40€"""
"""6,5 /10""","""Non""","""Noir""","""Android 11""",null,"""3""","""Chine""","""190,00 g""","""Pas de charge …","""2,740 W/kg""","""163,80 mm""","""OPPO""","""Micro SD""","""8,40 mm""","""5""","""75,60 mm""","""5000 mAh""","""OPPO A16""","""1,240 W/kg""","""2 SIM ou 1 SIM…","""4 Go""","""LCD""","""Nano SIM (x2) …","""4G""","""64 Go""","""8 coeurs jusqu…","""6,5"" soit 16,5…","""1600 x 720 pix…","""0,990 W/kg""","""212,94€"""
"""8,0 /10""","""Oui""","""Noir""","""Android 12""",null,"""4""","""Viêt Nam""","""195,00 g""","""Pas de charge …","""2,800 W/kg""","""165,10 mm""","""SAMSUNG""","""Micro SD""","""8,80 mm""","""5""","""76,40 mm""","""5000 mAh""","""Samsung Galaxy…","""1,390 W/kg""","""2 SIM""","""4 Go""","""TFT (LCD)""","""Dual SIM""","""4G""","""64 Go""","""8 coeurs jusqu…","""6,6"" soit 16,8…","""2408 x 1080 pi…","""0,367 W/kg""","""212,40€"""
"""6,5 /10""","""Non""","""Bleu""","""Android 11""",null,"""3""","""Chine""","""190,00 g""","""Pas de charge …","""2,740 W/kg""","""163,80 mm""","""OPPO""","""Micro SD""","""8,40 mm""","""5""","""75,60 mm""","""5000 mAh""","""OPPO A16""","""1,240 W/kg""","""2 SIM ou 1 SIM…","""4 Go""","""LCD""","""Nano SIM (x2) …","""4G""","""64 Go""","""8 coeurs jusqu…","""6,5"" soit 16,5…","""1600 x 720 pix…","""0,990 W/kg""","""211,68€"""


En revanche, ces 6 téléphones Android ont un usb type c, le non-renseignement de la part du reseller est étrange...

In [52]:
test = df.group_by(pl.col("model")).count().sort("count", descending=True).to_numpy()

In [125]:
df = df.with_columns(
    pl.when(pl.col("usb_type_c") == "Oui")
    .then(pl.lit(True))
    .when((pl.col("usb_type_c").is_null()) & (pl.col("brand") != "APPLE"))
    .then(pl.lit(True))
    .otherwise(pl.lit(False)) # les valeurs nulls sont chez apple, logique, ils n'ont pas d'usb c !
    .alias("usb_type_c")
)

In [126]:
df.filter(pl.col("brand") == "OPPO")

repairability_index,fast_charging,color,os,usb_type_c,sensor,made_in,net_weight,charging_type,das_limbs,height,brand,upgrade_storage,thickness,bluetooth,width,battery,model,das_chest,sim_number,ram,screen_tech,sim_type,network,storage,cpu,screen_size,screen_resolution,das_head,price,test
str,str,str,str,bool,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool
"""8,4 /10""","""Oui""","""Noir""","""Android 12""",true,"""3""","""Chine""","""218,00 g""","""Possibilité de…","""2,715 W/kg""","""163,70 mm""","""OPPO""","""Non""","""8,50 mm""","""5.2""","""73,90 mm""","""5000 mAh""","""OPPO Find X5 P…","""1,272 W/kg""","""2 SIM""","""12 Go""","""AMOLED Flexibl…","""Nano SIM et eS…","""5G""","""256 Go""","""8 coeurs jusqu…","""6,7"" soit 17 c…","""3216 x 1440 pi…","""0,989 W/kg""","""677,90€""",true
"""7,7 /10""","""Oui""","""Bleu""","""Android 12""",true,"""3""","""Chine""","""190,00 g""","""Pas de charge …","""2,683 W/kg""","""163,80 mm""","""OPPO""","""Micro SD""","""8,00 mm""","""5.2""","""75,10 mm""","""5000 mAh""","""OPPO A77""","""1,256 W/kg""","""2 SIM ou 1 SIM…","""4 Go""","""LCD HD+""","""Nano SIM (x2) …","""5G""","""64 Go""","""8 coeurs jusqu…","""6,5"" soit 16,5…","""1600 x 720 pix…","""0,986 W/kg""","""359,16€""",true
"""6,9 /10""","""Oui""","""Noir""","""Android 11""",true,"""4""","""Chine""","""193,00 g""","""Possibilité de…","""2,713 W/kg""","""163,60 mm""","""OPPO""","""Non""","""8,30 mm""","""5.2""","""74,00 mm""","""4500 mAh""","""OPPO Find X3 P…","""1,221 W/kg""","""2 SIM""","""12 Go""","""AMOLED (LTPO)""","""Nano SIM (x2) …","""5G""","""256 Go""","""8 coeurs jusqu…","""6,7"" soit 17 c…","""3216 x 1440 pi…","""0,881 W/kg""","""1099,94€""",true
"""7,9 /10""","""Oui""","""Noir""","""Android 11""",true,"""2""","""Chine""","""191,00 g""","""Pas de charge …","""2,640 W/kg""","""164,40 mm""","""OPPO""","""NM Card""","""8,40 mm""","""5""","""75,70 mm""","""5000 mAh""","""OPPO A96""","""1,220 W/kg""","""2 SIM ou 1 SIM…","""8 Go""","""FHD+""","""Nano SIM (x2) …","""4G""","""128 Go""","""Qualcomm Snapd…","""6,6"" soit 16,8…","""1080 x 2400 pi…","""0,810 W/kg""","""328,83€""",true
"""7,7 /10""","""Oui""","""Noir""","""Android 12""",true,"""3""","""Chine""","""190,00 g""","""Pas de charge …","""2,683 W/kg""","""163,80 mm""","""OPPO""","""Micro SD""","""8,00 mm""","""5.2""","""75,10 mm""","""5000 mAh""","""OPPO A77""","""1,256 W/kg""","""2 SIM ou 1 SIM…","""4 Go""","""LCD HD+""","""Nano SIM (x2) …","""5G""","""64 Go""","""8 coeurs jusqu…","""6,5"" soit 16,5…","""1600 x 720 pix…","""0,986 W/kg""","""255,11€""",true
"""8,5 /10""","""Oui""","""Noir""","""Android 11""",true,"""3""","""Chine""","""182,00 g""","""Pas de charge …","""2,776 W/kg""","""156,80 mm""","""OPPO""","""Non""","""7,60 mm""","""5.2""","""72,10 mm""","""4300 mAh""","""OPPO Reno 6""","""1,175 W/kg""","""2 SIM""","""8 Go""","""AMOLED Rigide""","""Nano SIM (x2)""","""5G""","""128 Go""","""8 coeurs jusqu…","""6,4"" soit 16,3…","""2400 x 1080 pi…","""0,957 W/kg""","""281,83€""",true
"""8,2 /10""","""Oui""","""Or""","""Android 12""",true,"""3""","""Chine""","""179,00 g""","""Pas de charge …","""2,756 W/kg""","""160,00 mm""","""OPPO""","""Non""","""7,70 mm""","""5.3""","""73,40 mm""","""4500 mAh""","""OPPO Reno 8""","""1,288 W/kg""","""2 SIM""","""8 Go""","""AMOLED Rigide""","""Nano SIM (x2)""","""5G""","""256 Go""","""8 coeurs jusqu…","""6,4"" soit 16,3…","""2400 x 1080 pi…","""0,960 W/kg""","""329,00€""",true
"""8,5 /10""","""Oui""","""Bleu""","""Android 11""",true,"""3""","""Chine""","""182,00 g""","""Pas de charge …","""2,776 W/kg""","""156,80 mm""","""OPPO""","""Non""","""7,60 mm""","""5.2""","""72,10 mm""","""4300 mAh""","""OPPO Reno 6""","""1,175 W/kg""","""2 SIM""","""8 Go""","""AMOLED Rigide""","""Nano SIM (x2)""","""5G""","""128 Go""","""8 coeurs jusqu…","""6,4"" soit 16,3…","""2400 x 1080 pi…","""0,957 W/kg""","""277,56€""",true
"""7,3 /10""","""Oui""","""Bleu""","""Android 10""",true,"""3""","""Chine""","""

In [81]:
df = df.with_columns(
    pl.col("price")
    .str.replace(",",".")
    .str.replace("€","")
    .cast(pl.Float64)
    )

In [82]:
df = df.with_columns(
    pl.col("repairability_index")
    .str.replace(",",".")
    .str.replace(" /10","")
    .cast(pl.Float64)
    )

In [85]:
df.select(
    pl.col(
        "model",
        "storage",
        "ram",
        "price",
        "repairability_index",
        "battery",
        "brand",
        "os",
        "color"
    )
).sort("price",descending=True)

model,storage,ram,price,repairability_index,battery,brand,os,color
str,str,str,f64,f64,str,str,str,str
"""Samsung Galaxy…","""1 To""","""12 Go""",2279.0,8.0,"""4400 mAh""","""SAMSUNG""","""Android 13""","""Beige"""
"""Samsung Galaxy…","""1 To""","""12 Go""",2279.0,8.0,"""4400 mAh""","""SAMSUNG""","""Android 13""","""Noir"""
"""Samsung Galaxy…","""1 To""","""12 Go""",2279.0,8.0,"""4400 mAh""","""SAMSUNG""","""Android 13""","""Noir"""
"""Samsung Galaxy…","""512 Go""","""12 Go""",2199.09,7.8,"""4400 mAh""","""SAMSUNG""","""Android 11""","""Noir"""
"""HUAWEI Mate X3…","""512 Go""","""12 Go""",2199.0,7.1,"""4800 mAh""","""HUAWEI""","""EMUI 13.1""","""Noir"""
"""HUAWEI Mate X3…","""512 Go""","""12 Go""",2084.98,7.1,"""4800 mAh""","""HUAWEI""","""EMUI 13.1""","""Vert"""
"""Samsung Galaxy…","""512 Go""","""12 Go""",2039.0,8.0,"""4400 mAh""","""SAMSUNG""","""Android 13""","""Noir"""
"""Samsung Galaxy…","""512 Go""","""12 Go""",2039.0,8.0,"""4400 mAh""","""SAMSUNG""","""Android 13""","""Bleu"""
"""Samsung Galaxy…","""512 Go""","""12 Go""",2039.0,8.0,"""4400 mAh""","""SAMSUNG""","""Android 13""","""Beige"""
